In [ ]:
"""
LIVE RANGE BREAKOUT BOT - MNQ
10:00-11:15 AM RANGE (75 minutes)
Matches backtested strategy exactly
IMPROVED LOGGING FOR 24/7 OPERATION
FIXED: Won't look for new setups while position is open
"""
import os
import requests
from dotenv import load_dotenv
from datetime import datetime, time, timedelta
import pytz
import time as time_module

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"
CONTRACT_ID = "CON.F.US.MNQ.H26"

# Strategy parameters
POSITION_SIZE = 1
FIXED_STOP_LOSS = 15  # points
FIXED_TAKE_PROFIT = 60  # points

# Safety parameters
MAX_DAILY_LOSS = -500  # Maximum loss per day
MAX_DAILY_PROFIT = 1500  # Maximum profit per day
PNL_CHECK_INTERVAL = 5  # Check P&L every 5 seconds when in trade

# Range parameters
RANGE_START_HOUR = 10
RANGE_START_MINUTE = 0
RANGE_END_HOUR = 11
RANGE_END_MINUTE = 15

# Trading hours (ET)
MARKET_OPEN = time(9, 30)
RANGE_COMPLETE = time(11, 15)
EOD_CLOSE = time(16, 0)

# Global state
range_high = None
range_low = None
in_position = False
daily_pnl = 0.0
starting_balance = 0.0
last_checked_bar = None
last_status_log = None  # Track last status message to avoid spam

# ==================== LOGGING ====================
def log(message, force=False):
    """Log with timestamp"""
    ny_tz = pytz.timezone('America/New_York')
    timestamp = datetime.now(ny_tz).strftime("%H:%M:%S")
    print(f"[{timestamp}] {message}")

def log_header(message):
    """Log a header"""
    print("\n" + "="*70)
    print(f"  {message}")
    print("="*70)

def log_status(message, interval_seconds=300):
    """Log status message only once per interval to avoid spam"""
    global last_status_log
    now = time_module.time()
    
    if last_status_log is None or (now - last_status_log) >= interval_seconds:
        log(message)
        last_status_log = now
        return True
    return False

# ==================== API FUNCTIONS ====================
def authenticate():
    """Authenticate and return headers"""
    try:
        resp = requests.post(
            f"{API_BASE}/Auth/loginKey",
            json={"userName": USERNAME, "apiKey": KEY},
            headers={"accept": "text/plain", "Content-Type": "application/json"},
            timeout=15
        )
        
        if resp.status_code != 200:
            log(f"❌ Authentication failed: {resp.status_code}")
            return None
        
        token = resp.json()["token"]
        return {
            "Authorization": f"Bearer {token}",
            "accept": "text/plain",
            "Content-Type": "application/json"
        }
    except Exception as e:
        log(f"❌ Authentication error: {e}")
        return None

def get_account(headers):
    """Get active account"""
    try:
        resp = requests.post(
            f"{API_BASE}/Account/search",
            json={"onlyActiveAccounts": True},
            headers=headers,
            timeout=15
        )
        
        accounts = resp.json().get("accounts", [])
        if not accounts:
            log("❌ No active accounts found")
            return None
        
        return accounts[0]
    except Exception as e:
        log(f"❌ Error getting account: {e}")
        return None

def get_current_positions(headers, account_id):
    """Get current MNQ positions"""
    try:
        resp = requests.post(
            f"{API_BASE}/Position/searchOpen",
            json={"accountId": account_id},
            headers=headers,
            timeout=15
        )
        
        if resp.status_code != 200:
            return []
        
        result = resp.json()
        if not result.get('success'):
            return []
        
        positions = result.get("positions", [])
        mnq_positions = [p for p in positions if p.get("contractId") == CONTRACT_ID]
        return mnq_positions
    except Exception as e:
        log(f"❌ Error getting positions: {e}")
        return []

def get_daily_pnl_from_balance(headers, starting_balance):
    """Calculate P&L from balance change - most reliable method"""
    try:
        account = get_account(headers)
        if not account:
            return 0.0
        
        current_balance = account.get('balance', starting_balance)
        pnl = current_balance - starting_balance
        
        return pnl
    except:
        return 0.0

def get_todays_range(headers):
    """Get today's 10:00-11:15 AM range"""
    try:
        ny_tz = pytz.timezone('America/New_York')
        now = datetime.now(ny_tz)
        
        start_time = now.replace(hour=9, minute=30, second=0, microsecond=0)
        end_time = now
        
        payload = {
            "contractId": CONTRACT_ID,
            "live": False,
            "startTime": start_time.astimezone(pytz.UTC).strftime("%Y-%m-%dT%H:%M:%SZ"),
            "endTime": end_time.astimezone(pytz.UTC).strftime("%Y-%m-%dT%H:%M:%SZ"),
            "unit": 2,
            "unitNumber": 5,
            "limit": 100,
            "includePartialBar": False
        }
        
        resp = requests.post(
            f"{API_BASE}/History/retrieveBars",
            json=payload,
            headers=headers,
            timeout=30
        )
        
        if resp.status_code != 200:
            return None, None
        
        response_data = resp.json()
        bars = response_data.get("bars")
        
        if not bars:
            return None, None
        
        # Filter bars within 10:00-11:15 range period
        range_bars = []
        for bar in bars:
            timestamp = bar['t'].replace('+00:00', 'Z')
            dt_utc = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
            dt_utc = pytz.utc.localize(dt_utc)
            dt_ny = dt_utc.astimezone(ny_tz)
            
            bar_time = dt_ny.time()
            range_start = time(RANGE_START_HOUR, RANGE_START_MINUTE)
            range_end = time(RANGE_END_HOUR, RANGE_END_MINUTE)
            
            if range_start <= bar_time < range_end:
                range_bars.append(bar)
        
        if not range_bars:
            return None, None
        
        high = max(bar['h'] for bar in range_bars)
        low = min(bar['l'] for bar in range_bars)
        
        return high, low
    except Exception as e:
        log(f"❌ Error calculating range: {e}")
        return None, None

def fetch_latest_bars(headers, num_bars=2):
    """Fetch the most recent completed 5-minute bars AFTER range period"""
    try:
        ny_tz = pytz.timezone('America/New_York')
        now = datetime.now(ny_tz)
        
        # FIXED: Only fetch bars AFTER 11:15 AM
        range_end = now.replace(hour=11, minute=15, second=0, microsecond=0)
        
        # If before range end, return empty
        if now < range_end:
            return []
        
        # Start from 11:15 AM
        start_time = range_end
        end_time = now
        
        payload = {
            "contractId": CONTRACT_ID,
            "live": False,
            "startTime": start_time.astimezone(pytz.UTC).strftime("%Y-%m-%dT%H:%M:%SZ"),
            "endTime": end_time.astimezone(pytz.UTC).strftime("%Y-%m-%dT%H:%M:%SZ"),
            "unit": 2,
            "unitNumber": 5,
            "limit": num_bars,
            "includePartialBar": False
        }
        
        resp = requests.post(
            f"{API_BASE}/History/retrieveBars",
            json=payload,
            headers=headers,
            timeout=30
        )
        
        bars = resp.json().get("bars", [])
        
        # Parse with NY timezone
        for bar in bars:
            timestamp = bar['t'].replace('+00:00', 'Z')
            dt_utc = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
            dt_utc = pytz.utc.localize(dt_utc)
            bar['dt_ny'] = dt_utc.astimezone(ny_tz)
            bar['time_str'] = bar['dt_ny'].strftime('%I:%M %p')
        
        # Reverse to chronological order
        bars = list(reversed(bars))
        
        return bars
    except Exception as e:
        log(f"❌ Error fetching bars: {e}")
        return []

def check_for_setup(bars):
    """Check if we have a valid setup"""
    global range_high, range_low, last_checked_bar
    
    if len(bars) < 2:
        return None, None
    
    if range_high is None or range_low is None:
        return None, None
    
    bar_1 = bars[-2]
    bar_2 = bars[-1]
    
    if last_checked_bar == bar_2['t']:
        return None, None
    
    last_checked_bar = bar_2['t']
    
    bar_1_close = bar_1['c']
    bar_2_close = bar_2['c']
    
    log(f"📊 Bar 1 ({bar_1['time_str']}): {bar_1_close:.2f}")
    log(f"📊 Bar 2 ({bar_2['time_str']}): {bar_2_close:.2f}")
    
    # SHORT setup
    if bar_1_close > range_high and range_low <= bar_2_close <= range_high:
        log(f"🔴 BREAKOUT ABOVE: Bar 1 = {bar_1_close:.2f} (> {range_high:.2f})")
        log(f"🔵 PULLBACK INSIDE: Bar 2 = {bar_2_close:.2f}")
        log("🎯 SHORT SETUP DETECTED")
        return 'short', bar_2_close
    
    # LONG setup
    if bar_1_close < range_low and range_low <= bar_2_close <= range_high:
        log(f"🔴 BREAKOUT BELOW: Bar 1 = {bar_1_close:.2f} (< {range_low:.2f})")
        log(f"🔵 PULLBACK INSIDE: Bar 2 = {bar_2_close:.2f}")
        log("🎯 LONG SETUP DETECTED")
        return 'long', bar_2_close
    
    log("⚪ No setup detected")
    return None, None

def place_order(headers, account_id, setup_type):
    """Place order with SL/TP brackets"""
    try:
        side = 0 if setup_type == 'long' else 1
        
        if setup_type == 'long':
            sl_ticks = -(FIXED_STOP_LOSS * 4)
            tp_ticks = FIXED_TAKE_PROFIT * 4
        else:
            sl_ticks = FIXED_STOP_LOSS * 4
            tp_ticks = -(FIXED_TAKE_PROFIT * 4)
        
        order_payload = {
            "accountId": account_id,
            "contractId": CONTRACT_ID,
            "type": 2,
            "side": side,
            "size": POSITION_SIZE,
            "stopLossBracket": {
                "ticks": sl_ticks,
                "type": 4
            },
            "takeProfitBracket": {
                "ticks": tp_ticks,
                "type": 1
            }
        }
        
        log(f"📤 Placing {setup_type.upper()} order: {POSITION_SIZE} contracts")
        log(f"   SL: {FIXED_STOP_LOSS}pts | TP: {FIXED_TAKE_PROFIT}pts")
        
        resp = requests.post(
            f"{API_BASE}/Order/place",
            json=order_payload,
            headers=headers,
            timeout=15
        )
        
        result = resp.json()
        
        if result.get('success'):
            order_id = result.get('orderId')
            log(f"✅ Order placed successfully - ID: {order_id}")
            return order_id
        else:
            log(f"❌ Order failed: {result.get('errorMessage')}")
            return None
    except Exception as e:
        log(f"❌ Error placing order: {e}")
        return None

def close_all_positions_and_orders(headers, account_id):
    """Close MNQ position and cancel all orders"""
    try:
        log("🛑 Closing all positions and orders...")
        
        close_payload = {
            "accountId": account_id,
            "contractId": CONTRACT_ID
        }
        
        close_resp = requests.post(
            f"{API_BASE}/Position/closeContract",
            json=close_payload,
            headers=headers,
            timeout=15
        )
        
        close_result = close_resp.json()
        if close_result.get('success'):
            log("✅ Position closed")
        
        orders_resp = requests.post(
            f"{API_BASE}/Order/searchOpen",
            json={"accountId": account_id},
            headers=headers,
            timeout=15
        )
        
        if orders_resp.status_code == 200:
            orders_data = orders_resp.json()
            if orders_data.get('success'):
                open_orders = orders_data.get("orders", [])
                
                for order in open_orders:
                    cancel_payload = {
                        "accountId": account_id,
                        "orderId": order.get("id")
                    }
                    
                    requests.post(
                        f"{API_BASE}/Order/cancel",
                        json=cancel_payload,
                        headers=headers,
                        timeout=15
                    )
                
                if open_orders:
                    log(f"✅ Cancelled {len(open_orders)} order(s)")
        
        return True
    except Exception as e:
        log(f"❌ Error closing positions: {e}")
        return False

def get_time_until(target_time):
    """Get human-readable time until target"""
    ny_tz = pytz.timezone('America/New_York')
    now = datetime.now(ny_tz)
    
    target_dt = now.replace(hour=target_time.hour, minute=target_time.minute, second=0, microsecond=0)
    if target_dt < now:
        target_dt += timedelta(days=1)
    
    delta = target_dt - now
    hours = delta.seconds // 3600
    minutes = (delta.seconds % 3600) // 60
    
    if hours > 0:
        return f"{hours}h {minutes}m"
    else:
        return f"{minutes}m"

# ==================== MAIN BOT LOOP ====================
def main():
    global daily_pnl, in_position, range_high, range_low, last_checked_bar, starting_balance, last_status_log
    
    log_header("LIVE RANGE BREAKOUT BOT - 24/7 MODE")
    log(f"📊 Strategy: {POSITION_SIZE} contracts | {FIXED_STOP_LOSS}pt SL | {FIXED_TAKE_PROFIT}pt TP")
    log(f"⏰ Range: {RANGE_START_HOUR:02d}:{RANGE_START_MINUTE:02d}-{RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d} (75 min)")
    log(f"💰 Daily Limits: ${MAX_DAILY_LOSS} loss / +${MAX_DAILY_PROFIT} profit")
    log(f"🕐 Trading Hours: {MARKET_OPEN.strftime('%I:%M %p')} - {EOD_CLOSE.strftime('%I:%M %p')} ET")
    
    ny_tz = pytz.timezone('America/New_York')
    
    # Authenticate
    log("\n🔐 Authenticating...")
    headers = authenticate()
    if not headers:
        log("❌ Failed to authenticate. Exiting.")
        return
    log("✅ Authenticated")
    
    # Get account
    account = get_account(headers)
    if not account:
        log("❌ Failed to get account. Exiting.")
        return
    
    account_id = account["id"]
    starting_balance = account.get('balance', 0.0)
    log(f"👤 Account: {account['name']}")
    log(f"💵 Starting Balance: ${starting_balance:,.2f}")
    
    current_trading_day = None
    daily_limit_hit = False
    
    log("\n🚀 Bot running - Ctrl+C to stop\n")
    
    # Main loop
    while True:
        try:
            now = datetime.now(ny_tz)
            current_time = now.time()
            today = now.date()
            day_name = now.strftime('%A')
            
            # Weekend check
            if now.weekday() >= 5:
                time_until_monday = get_time_until(MARKET_OPEN)
                log_status(f"💤 WEEKEND ({day_name}) - Market closed. Opens Monday at {MARKET_OPEN.strftime('%I:%M %p')} ET ({time_until_monday})")
                time_module.sleep(3600)
                continue
            
            # New trading day
            if current_trading_day != today:
                log_header(f"📅 NEW TRADING DAY: {today.strftime('%A, %B %d, %Y')}")
                
                current_trading_day = today
                daily_pnl = 0.0
                in_position = False
                range_high = None
                range_low = None
                last_checked_bar = None
                daily_limit_hit = False
                last_status_log = None
                
                # Reset starting balance
                account = get_account(headers)
                if account:
                    starting_balance = account.get('balance', starting_balance)
                    log(f"💵 Starting Balance: ${starting_balance:,.2f}")
                
                log("✅ Daily variables reset")
                
                headers = authenticate()
                if not headers:
                    log("❌ Re-authentication failed")
                    time_module.sleep(300)
                    continue
            
            # Hourly re-auth
            if now.minute == 0 and now.second < 10:
                headers = authenticate()
                if headers:
                    log("🔐 Re-authenticated successfully")
                else:
                    log("❌ Re-authentication failed")
                    time_module.sleep(60)
                    continue
            
            # If limit hit, sleep
            if daily_limit_hit:
                log_status(f"🛑 Daily limit reached - Done trading for today. P&L: ${daily_pnl:.2f}")
                time_module.sleep(3600)
                continue
            
            # Pre-market
            if current_time < MARKET_OPEN:
                time_until_open = get_time_until(MARKET_OPEN)
                log_status(f"⏳ PRE-MARKET - Market opens at {MARKET_OPEN.strftime('%I:%M %p')} ET in {time_until_open}")
                time_module.sleep(60)
                continue
            
            # After close
            if current_time >= EOD_CLOSE:
                if current_time.hour == 16 and current_time.minute == 0 and current_time.second < 10:
                    positions = get_current_positions(headers, account_id)
                    if positions:
                        log("🔔 4:00 PM ET - Closing all positions (EOD)")
                        close_all_positions_and_orders(headers, account_id)
                    
                    log_header("📊 MARKET CLOSED")
                    log(f"Final P&L: ${daily_pnl:.2f}")
                    log(f"Next market open: {(now + timedelta(days=1)).strftime('%A')} at {MARKET_OPEN.strftime('%I:%M %p')} ET")
                else:
                    log_status(f"🌙 AFTER HOURS - Market closed. Final P&L: ${daily_pnl:.2f}")
                
                time_module.sleep(3600)
                continue
            
            # Range formation period
            if current_time < RANGE_COMPLETE:
                time_until_complete = get_time_until(RANGE_COMPLETE)
                log_status(f"📏 RANGE FORMING - Window: {RANGE_START_HOUR:02d}:{RANGE_START_MINUTE:02d}-{RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d} ET | Complete in {time_until_complete}")
                time_module.sleep(30)
                continue
            
            # Calculate range (once at 11:15 AM or later)
            if range_high is None and current_time >= RANGE_COMPLETE:
                log_header("📐 CALCULATING RANGE")
                range_high, range_low = get_todays_range(headers)
                
                if range_high and range_low:
                    range_size = range_high - range_low
                    log(f"✅ Range High: {range_high:.2f}")
                    log(f"✅ Range Low:  {range_low:.2f}")
                    log(f"📏 Range Size: {range_size:.2f} points")
                    log(f"⏳ Waiting for bars after {RANGE_COMPLETE.strftime('%I:%M %p')} ET to check for setups...")
                else:
                    log("❌ Failed to calculate range - Retrying in 60s...")
                    time_module.sleep(60)
                    continue
            
            # Update P&L from balance
            daily_pnl = get_daily_pnl_from_balance(headers, starting_balance)
            
            # Check loss limit
            if daily_pnl <= MAX_DAILY_LOSS:
                log_header("🛑 DAILY LOSS LIMIT REACHED")
                log(f"P&L: ${daily_pnl:.2f} (Limit: ${MAX_DAILY_LOSS})")
                close_all_positions_and_orders(headers, account_id)
                daily_limit_hit = True
                continue
            
            # Check profit limit
            if daily_pnl >= MAX_DAILY_PROFIT:
                log_header("🎉 DAILY PROFIT TARGET REACHED")
                log(f"P&L: ${daily_pnl:.2f} (Target: ${MAX_DAILY_PROFIT})")
                close_all_positions_and_orders(headers, account_id)
                daily_limit_hit = True
                continue
            
            # CRITICAL: Check position status FIRST - don't look for setups if in position
            positions = get_current_positions(headers, account_id)
            was_in_position = in_position
            in_position = len(positions) > 0
            
            # Position just closed
            if was_in_position and not in_position:
                log_header("✅ POSITION CLOSED")
                log("Position closed (TP or SL hit)")
                daily_pnl = get_daily_pnl_from_balance(headers, starting_balance)
                log(f"💰 Updated P&L: ${daily_pnl:.2f}")
                log(f"🔍 Resuming setup monitoring...")
            
            # CRITICAL: If in position, ONLY monitor - DO NOT look for new setups
            if in_position:
                if now.second % PNL_CHECK_INTERVAL == 0:
                    log(f"🔄 IN POSITION | P&L: ${daily_pnl:.2f} | Loss Limit: ${MAX_DAILY_LOSS} | Profit Target: ${MAX_DAILY_PROFIT}")
                time_module.sleep(1)
                continue  # SKIP ALL SETUP CHECKING
            
            # Only look for setups if:
            # 1. Range is calculated
            # 2. NOT in position
            # 3. After 11:15 AM
            if range_high is not None and not in_position:
                if now.second % 30 == 0:  # Check every 30 seconds
                    bars = fetch_latest_bars(headers, num_bars=10)
                    
                    if len(bars) < 2:
                        # Not enough bars yet
                        log_status(f"⏳ Waiting for bars after {RANGE_COMPLETE.strftime('%I:%M %p')} ET (need 2+ bars, have {len(bars)})")
                        time_module.sleep(1)
                        continue
                    
                    log_header(f"🔍 CHECKING FOR SETUPS - {now.strftime('%I:%M %p')} ET")
                    log(f"💰 Daily P&L: ${daily_pnl:.2f}")
                    log(f"📊 Range: {range_low:.2f} - {range_high:.2f}")
                    log(f"📈 Bars available: {len(bars)} (from {bars[0]['time_str']} to {bars[-1]['time_str']})")
                    
                    setup_type, entry_price = check_for_setup(bars)
                    
                    if setup_type:
                        order_id = place_order(headers, account_id, setup_type)
                        
                        if order_id:
                            in_position = True
                            log(f"🚀 Entered {setup_type.upper()} position")
                            log(f"⏸️  Pausing setup monitoring until position closes")
                    
                    time_module.sleep(5)
                else:
                    time_module.sleep(1)
            else:
                time_module.sleep(1)
        
        except KeyboardInterrupt:
            log_header("🛑 BOT STOPPED BY USER")
            log(f"Final P&L: ${daily_pnl:.2f}")
            break
        except Exception as e:
            log(f"❌ ERROR in main loop: {e}")
            import traceback
            traceback.print_exc()
            time_module.sleep(30)
    
    log_header("👋 BOT SHUTDOWN COMPLETE")

if __name__ == "__main__":
    main()


  LIVE RANGE BREAKOUT BOT - 24/7 MODE
[13:16:29] 📊 Strategy: 1 contracts | 15pt SL | 60pt TP
[13:16:29] ⏰ Range: 10:00-11:15 (75 min)
[13:16:29] 💰 Daily Limits: $-500 loss / +$1500 profit
[13:16:29] 🕐 Trading Hours: 09:30 AM - 04:00 PM ET
[13:16:29] 
🔐 Authenticating...
[13:16:29] ✅ Authenticated
[13:16:29] 👤 Account: 50KTC-V2-311503-72551517
[13:16:29] 💵 Starting Balance: $49,375.74
[13:16:29] 
🚀 Bot running - Ctrl+C to stop


  📅 NEW TRADING DAY: Thursday, February 05, 2026
[13:16:29] 💵 Starting Balance: $49,375.74
[13:16:29] ✅ Daily variables reset

  📐 CALCULATING RANGE
[13:16:30] ✅ Range High: 24805.50
[13:16:30] ✅ Range Low:  24537.00
[13:16:30] 📏 Range Size: 268.50 points
[13:16:30] ⏳ Waiting for bars after 11:15 AM ET to check for setups...

  🔍 CHECKING FOR SETUPS - 01:17 PM ET
[13:17:30] 💰 Daily P&L: $0.00
[13:17:30] 📊 Range: 24537.00 - 24805.50
[13:17:30] 📈 Bars available: 10 (from 12:25 PM to 01:10 PM)
[13:17:30] 📊 Bar 1 (01:05 PM): 24766.75
[13:17:30] 📊 Bar 2 (01:10 PM): 